# Labelling Algorithm
The intention of this notebook is, to learn how to use C Code in Python and with that knowledge implement a labelling algorithm for the CVRP in C.

## CFFI Tutorial
https://medium.com/@torbengraebergt/c-c-and-python-class-interfacing-f45a9ee352f4

In [1]:
from cffi import FFI

dllURI = "labelling_lib.so"

ffi = FFI()
labelling_lib = ffi.dlopen(dllURI)

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity); void labelling(double const * dual,const bool farkas, unsigned* result);"
ffi.cdef(funDefs, override=True)

## Passing a Graph to the C Code

In [2]:
import networkx as nx
import random
import numpy as np

In [3]:
G = nx.complete_graph(10)
for (u,v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
# G.edges()[7,9]['weight'] = 100
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

In [4]:
node_data = list(nx.get_node_attributes(G,"demand").values())

nodes_arr = ffi.cast("unsigned*", np.array(node_data).astype(np.uintc).ctypes.data)
# TODO: Extract raw matrix from G and pass it to initGraph
edges = nx.adjacency_matrix(G,dtype=np.double).toarray()
edges_arr = ffi.cast("double*", edges.ctypes.data)
num_nodes = ffi.cast("unsigned",G.number_of_nodes())

capacity = 30
capacity_ptr = ffi.cast("double",capacity)

result = np.zeros(capacity + 2,dtype=np.uintc)
result_arr = ffi.cast("unsigned*",result.ctypes.data)



/var/folders/tv/pzvp6rwx029b6p4rgqv2cx0h0000gn/T/ipykernel_9656/3769196214.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  edges = nx.adjacency_matrix(G,dtype=np.double).toarray()


In [8]:
labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr)

dual = np.array(range(10)).astype(np.double)
dual = np.full(10,1.8,dtype=np.double)
pointer_dual = ffi.cast("double*", dual.ctypes.data)

labelling_lib.labelling(pointer_dual, True,result_arr)

print(result)


[0 8 3 8 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]Graph data successfully copied to C.

Found Path with negative reduced cost
Current label is 8 and pred is 3
Current label is 3 and pred is 8
Current label is 8 and pred is 5
